In [1]:
import cv2
import json
import os
import numpy as np
import copy

In [2]:
# def calculate_bounding_box_area(bbox):
#     """Calculate the area of a bounding box: width * height."""
#     return bbox["width"] * bbox["height"]

# def calculate_bounding_box_center(bbox):
#     """Calculate the center of a bounding box (x, y)."""
#     center_x = bbox["left"] + bbox["width"] / 2
#     center_y = bbox["top"] + bbox["height"] / 2
#     return center_x, center_y

# def find_outlier_images(final_annotations_file, area_threshold=3, center_threshold=2):
#     with open(final_annotations_file, 'r') as f:
#         final_annotations = json.load(f)
    
#     image_bounding_box_sizes = {}
#     image_centers = {}
#     all_bounding_box_sizes = []
#     all_centers = []
    
#     for image_name, image_data in final_annotations["images"].items():
#         total_area = 0
#         centers = []

#         for part in image_data["available_parts"]:
#             bbox = part["absolute_bounding_box"]
#             area = calculate_bounding_box_area(bbox)
#             total_area += area
#             center = calculate_bounding_box_center(bbox)
#             centers.append(center)

#         image_bounding_box_sizes[image_name] = total_area
#         image_centers[image_name] = np.mean(centers, axis=0)
#         all_bounding_box_sizes.append(total_area)
#         all_centers.extend(centers)
    
#     mean_area = np.mean(all_bounding_box_sizes)
#     std_area = np.std(all_bounding_box_sizes)
    
#     mean_center = np.mean(all_centers, axis=0)
#     center_distances = np.linalg.norm(np.array(all_centers) - mean_center, axis=1)
#     std_center = np.std(center_distances)

#     outliers = []
#     for image_name in image_bounding_box_sizes:
#         area = image_bounding_box_sizes[image_name]
#         center = image_centers[image_name]
#         center_distance = np.linalg.norm(center - mean_center)
        
#         if abs(area - mean_area) > area_threshold * std_area or center_distance > center_threshold * std_center:
#             outliers.append(image_name)
    
#     print(f"Outlier images identified: {outliers}")
#     return outliers

In [ ]:
HANDLE_SIZE = 2
is_ctrl_pressed = False
pan_offset = [0, 0]
last_mouse_pos = [0, 0]
panning = False
undo_stack = []
selected_box = None
corner_drag = None
mode = {"drag": False, "draw_start": None, "offset": (0, 0)}
is_zoomed = False

def point_in_rect(x, y, rect):
    return rect[0] <= x <= rect[0] + rect[2] and rect[1] <= y <= rect[1] + rect[3]

def get_handle(box, x, y):
    handles = {
        'tl': (box["left"], box["top"]),
        'tr': (box["left"] + box["width"], box["top"]),
        'bl': (box["left"], box["top"] + box["height"]),
        'br': (box["left"] + box["width"], box["top"] + box["height"])
    }
    for name, (hx, hy) in handles.items():
        if abs(x - hx) <= HANDLE_SIZE and abs(y - hy) <= HANDLE_SIZE:
            return name
    return None

def draw_boxes(img, boxes):
    img_disp = img.copy()
    for box in boxes:
        x, y, w, h = box["left"], box["top"], box["width"], box["height"]
        cv2.rectangle(img_disp, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img_disp, box.get("part_name", ""), (x, y - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        for hx, hy in [(x, y), (x + w, y), (x, y + h), (x + w, y + h)]:
            cv2.rectangle(img_disp, (hx - HANDLE_SIZE // 2, hy - HANDLE_SIZE // 2),
                          (hx + HANDLE_SIZE // 2, hy + HANDLE_SIZE // 2), (255, 0, 0), -1)
    return img_disp

def mouse_callback(event, x, y, flags, param):
    global selected_box, corner_drag, mode, pan_offset, last_mouse_pos, panning, undo_stack, is_ctrl_pressed, is_zoomed

    boxes = param["boxes"]
    if is_zoomed:
        if event == cv2.EVENT_LBUTTONDOWN:
            for idx, box in enumerate(boxes):
                handle = get_handle(box, x - pan_offset[0], y - pan_offset[1])
                if handle:
                    selected_box = idx
                    corner_drag = handle
                    return
            for idx, box in enumerate(boxes):
                if point_in_rect(x - pan_offset[0], y - pan_offset[1],
                                 (box["left"], box["top"], box["width"], box["height"])):
                    selected_box = idx
                    mode["offset"] = (x - pan_offset[0] - box["left"], y - pan_offset[1] - box["top"])
                    mode["drag"] = True
                    return
            selected_box = None
        return

    if event == cv2.EVENT_LBUTTONDOWN:
        for idx, box in enumerate(boxes):
            handle = get_handle(box, x - pan_offset[0], y - pan_offset[1])
            if handle:
                selected_box = idx
                corner_drag = handle
                return
        for idx, box in enumerate(boxes):
            if point_in_rect(x - pan_offset[0], y - pan_offset[1],
                             (box["left"], box["top"], box["width"], box["height"])):
                selected_box = idx
                mode["offset"] = (x - pan_offset[0] - box["left"], y - pan_offset[1] - box["top"])
                mode["drag"] = True
                return
        selected_box = None

    elif event == cv2.EVENT_MOUSEMOVE:
        if corner_drag and selected_box is not None:
            box = boxes[selected_box]
            px, py = x - pan_offset[0], y - pan_offset[1]
            if corner_drag == 'tl':
                box["width"] += box["left"] - px
                box["height"] += box["top"] - py
                box["left"], box["top"] = px, py
            elif corner_drag == 'tr':
                box["width"] = px - box["left"]
                box["height"] += box["top"] - py
                box["top"] = py
            elif corner_drag == 'bl':
                box["width"] += box["left"] - px
                box["left"] = px
                box["height"] = py - box["top"]
            elif corner_drag == 'br':
                box["width"] = px - box["left"]
                box["height"] = py - box["top"]
            update_display_image(param)

        elif mode.get("drag") and selected_box is not None:
            dx, dy = mode["offset"]
            box = boxes[selected_box]
            box["left"] = x - pan_offset[0] - dx
            box["top"] = y - pan_offset[1] - dy
            update_display_image(param)

    elif event == cv2.EVENT_LBUTTONUP:
        mode["drag"] = False
        corner_drag = None

    elif event == cv2.EVENT_RBUTTONDOWN:
        mode["draw_start"] = (x - pan_offset[0], y - pan_offset[1])

    elif event == cv2.EVENT_RBUTTONUP:
        x0, y0 = mode["draw_start"]
        x1, y1 = x - pan_offset[0], y - pan_offset[1]
        new_box = {
            "part_name": f"part_{len(boxes)}",
            "left": min(x0, x1),
            "top": min(y0, y1),
            "width": abs(x1 - x0),
            "height": abs(y1 - y0),
        }
        boxes.append(new_box)
        undo_stack.append(copy.deepcopy(boxes))
        update_display_image(param)

def update_display_image(param):
    global pan_offset
    img_with_boxes = draw_boxes(param["original_img"], param["boxes"])
    translated = np.zeros_like(img_with_boxes)
    x_offset, y_offset = pan_offset
    h, w = img_with_boxes.shape[:2]

    x_start = max(0, x_offset)
    y_start = max(0, y_offset)
    x_end = min(w, w + x_offset)
    y_end = min(h, h + y_offset)

    translated[y_start:y_end, x_start:x_end] = img_with_boxes[max(0, -y_offset):h - max(0, y_offset),
                                                               max(0, -x_offset):w - max(0, x_offset)]
    param["img"][:] = translated

def edit_bounding_boxes_by_index(annotations_path, image_dir, indices_to_edit, max_images=None):
    global selected_box, corner_drag, mode, is_zoomed, undo_stack

    with open(annotations_path, 'r') as f:
        annotations = json.load(f)

    all_image_names = list(annotations["images"].keys())
    selected_images = [all_image_names[i] for i in indices_to_edit if i < len(all_image_names)]
    if max_images is not None:
        selected_images = selected_images[:max_images]

    for i, image_name in enumerate(selected_images, start=1):
        selected_box = None
        corner_drag = None
        mode = {"drag": False, "draw_start": None, "offset": (0, 0)}
        is_zoomed = False

        image_path = os.path.join(image_dir, image_name)
        if not os.path.exists(image_path):
            print(f"Image {image_name} not found, skipping...")
            continue

        print(f"\nEditing ({i}/{len(selected_images)}): {image_name} — ESC to save, M to mirror, Z to undo")
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        original_img = img.copy()

        boxes = annotations["images"][image_name]["available_parts"]
        for box in boxes:
            box.update(box["absolute_bounding_box"])
        undo_stack.clear()
        undo_stack.append(copy.deepcopy(boxes))

        display_img = draw_boxes(original_img, boxes)
        cv2.namedWindow("Edit Bounding Boxes")
        cv2.setMouseCallback("Edit Bounding Boxes", mouse_callback, {
            "boxes": boxes,
            "img": display_img,
            "original_img": original_img
        })

        is_mirrored = False

        while True:
            img_show = display_img.copy()
            cv2.putText(img_show, f"{i}/{len(selected_images)}", (10, 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.imshow("Edit Bounding Boxes", img_show)
            key = cv2.waitKey(1)

            if key == 27:
                if is_mirrored:
                    save_path = os.path.join(image_dir, image_name)
                    mirrored_bgr = cv2.cvtColor(original_img, cv2.COLOR_RGB2BGR)
                    cv2.imwrite(save_path, mirrored_bgr)
                break

            elif key == ord('m'):
                is_mirrored = not is_mirrored
                original_img = cv2.flip(original_img, 1)
                display_img[:] = draw_boxes(original_img, boxes)

            elif key == ord('z'):
                if len(undo_stack) > 1:
                    undo_stack.pop()
                    boxes[:] = copy.deepcopy(undo_stack[-1])
                    update_display_image({
                        "boxes": boxes,
                        "img": display_img,
                        "original_img": original_img
                    })
                    print("Undo successful")
                else:
                    print("Nothing to undo")

        cv2.destroyAllWindows()

        for j, box in enumerate(boxes):
            annotations["images"][image_name]["available_parts"][j]["absolute_bounding_box"] = {
                "left": int(box["left"]),
                "top": int(box["top"]),
                "width": int(box["width"]),
                "height": int(box["height"]),
            }

        with open(annotations_path, 'w') as f:
            json.dump(annotations, f, indent=2)

        print(f"Saved updated boxes for {image_name}")

annotations_file = '../../data/processed/final_annotations.json'
image_folder = '../../data/images'

indices_to_edit = list(range(7829, 7976))
edit_bounding_boxes_by_index(annotations_file, image_folder, indices_to_edit)


Editing (1/147): G0983-648473.jpg — ESC to save, M to mirror, Z to undo
Saved updated boxes for G0983-648473.jpg

Editing (2/147): G0983-648478.jpg — ESC to save, M to mirror, Z to undo
Saved updated boxes for G0983-648478.jpg

Editing (3/147): G0983-648517.jpg — ESC to save, M to mirror, Z to undo
Saved updated boxes for G0983-648517.jpg

Editing (4/147): G0983-918277.jpg — ESC to save, M to mirror, Z to undo
Saved updated boxes for G0983-918277.jpg

Editing (5/147): G0983-918287.jpg — ESC to save, M to mirror, Z to undo
Saved updated boxes for G0983-918287.jpg

Editing (6/147): G0983-918295.jpg — ESC to save, M to mirror, Z to undo
Saved updated boxes for G0983-918295.jpg

Editing (7/147): G0983-918307.jpg — ESC to save, M to mirror, Z to undo
Saved updated boxes for G0983-918307.jpg

Editing (8/147): G0983-924074.jpg — ESC to save, M to mirror, Z to undo
Saved updated boxes for G0983-924074.jpg

Editing (9/147): G0983-924077.jpg — ESC to save, M to mirror, Z to undo
Saved updated b

KeyboardInterrupt: 

: 